In [ ]:

import numpy as np
%matplotlib inline  

import argparse
import seaborn as sns
import  sys
import pytorch_lightning as pl
import torch
from tbsim.utils.log_utils import PrintLogger
from tbsim.utils.batch_utils import set_global_batch_type
from tbsim.utils.trajdata_utils import set_global_trajdata_batch_env, set_global_trajdata_batch_raster_cfg
import tbsim.utils.train_utils as TrainUtils
from tbsim.datasets.factory import datamodule_factory
from tbsim.utils.env_utils import RolloutCallback

import wandb,json
from pytorch_lightning.loggers import  WandbLogger
from  models.algos import  UnifiedTrainer
from datetime import  datetime
from configs.custom_config import dict_to_config,ConfigBase,serialize_object
from src.tbsim.configs.base import ExperimentConfig
import yaml
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict

from torch.utils.data import DataLoader
from tqdm import tqdm

from trajdata import AgentBatch, AgentType, UnifiedDataset
from trajdata.augmentation import NoiseHistories
from trajdata.visualization.vis import plot_agent_batch

parser = argparse.ArgumentParser(description="Training Script")
parser.add_argument("--config", type=str, default="/home/visier/hazardforge/HazardForge/config.yaml", help="Path to YAML config")

# 直接使用默认值解析参数
args = parser.parse_args([])  # 在 Notebook 中使用 [] 表示不传递命令行参数

# 加载配置文件
with open(args.config, "r") as f:
    config_dict = yaml.safe_load(f)
train_config = dict_to_config(ConfigBase, config_dict.get("train", {}))
env_config = dict_to_config(ConfigBase, config_dict.get("env", {}))
algo_config = dict_to_config(ConfigBase, config_dict.get("algo", {}))
default_config = ExperimentConfig(
        train_config=train_config,
        env_config=env_config,
        algo_config=algo_config,
        registered_name=config_dict.get("registered_name", "default_experiment"),
    )
if default_config.train.rollout.get("enabled", False):
        default_config.env["eval"] = {"env": default_config.env["name"]}
        assert default_config.algo["eval_class"], f"Please set an eval_class for {default_config.algo['name']}"
        default_config.env["eval"]["eval_class"] = default_config.algo["eval_class"]
        default_config.env["eval"]["dataset_path"] = default_config.train["trajdata_data_dirs"]["nusc_trainval"]
        env_specific_config = default_config.env.get(default_config.env["eval"]["env"], {})
        for key, value in env_specific_config.items():
            default_config.env["eval"][key] = value
default_config.lock()
cfg = default_config
pl.seed_everything(cfg.seed)
set_global_batch_type("trajdata")
set_global_trajdata_batch_env(cfg.train.trajdata_source_train[0])
set_global_trajdata_batch_raster_cfg(cfg.env.rasterizer)
print("\n============= New Training Run with Config =============")
datamodule = datamodule_factory(
        cls_name=cfg.train.datamodule_class, config=cfg
    )

datamodule.setup()
model = UnifiedTrainer(algo_config=cfg.algo,train_config=cfg.train,
                           modality_shapes=datamodule.modality_shapes,
                           registered_name=cfg.registered_name,
                           train_mode=cfg.train.mode)

In [4]:
from tbsim.utils.batch_utils import batch_utils
train_loader = datamodule.train_dataloader()
batch = next(iter(train_loader))
batch = batch_utils().parse_batch(batch)


In [ ]:
idx=10
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
from IPython.display import HTML

image = batch['image'][idx]  # (34, 224, 224)
sem_channel1 = image[-1].detach().cpu().numpy()
sem_channel2 = image[-2].detach().cpu().numpy()
sem_channel3 = image[-3].detach().cpu().numpy()

def normalize(x):
    x = x - x.min()
    if x.max() > 0:
        x = x / x.max()
    return x

r = normalize(sem_channel1)
g = normalize(sem_channel2)
b = normalize(sem_channel3)

rgb = np.stack([r,g,b], axis=-1) 
rgb = rgb*0.4+0.6

history_length = image.shape[0] - 3  
hist_image = image[:history_length].detach().cpu().numpy()  # shape: (history_length, H, W)


fig, ax = plt.subplots()

alpha = 0.5

def frame_to_image(frame_data):
    overlay = np.zeros_like(rgb)
    overlay[frame_data == 1] = [0, 1, 0]
    overlay[frame_data == -1] = [1, 0, 0]
    combined = rgb * (1 - alpha) + overlay * alpha
    return combined


initial_combined = frame_to_image(hist_image[0])
im = ax.imshow(initial_combined, animated=True)
ax.axis('off')

def update(frame):
    current_combined = frame_to_image(hist_image[frame])
    im.set_array(current_combined)
    return [im]

ani = animation.FuncAnimation(fig, update, frames=range(history_length), interval=200, blit=True)

HTML(ani.to_jshtml())


In [ ]:
drivable_map = batch['drivable_map'][idx].detach().cpu().numpy()
maps = batch['maps'][idx].detach().cpu().numpy().transpose(1, 2, 0)*0.5+0.5
# maps = maps * drivable_map[..., None]
fig, axes = plt.subplots(1, 3, figsize=(10, 5))
axes[0].imshow(drivable_map, cmap='gray')
axes[1].imshow(maps)
axes[2].imshow(maps * drivable_map[..., None])


In [ ]:
def visualize(batch,idx):
    maps = batch['maps'][idx].detach().cpu().numpy()
    maps_rgb = maps.transpose(1, 2, 0)
    maps_rgb=maps_rgb*0.4+0.7

    traj_state = batch['target_positions'].detach().cpu().numpy()
    raster_from_agent = batch['raster_from_agent'].detach().cpu().numpy()
    B, T, _ = traj_state.shape
    ones = np.ones((B, T, 1))
    traj_homo = np.concatenate([traj_state, ones], axis=-1)
    traj_homo_T = np.transpose(traj_homo, (0, 2, 1))
    traj_raster = np.matmul(raster_from_agent, traj_homo_T)
    traj_raster = np.transpose(traj_raster, (0, 2, 1))
    traj_raster_xy = traj_raster[..., :2]
    traj_xy = traj_raster_xy[idx]

    
    plt.imshow(maps_rgb)
    plt.plot(traj_xy[:, 0], traj_xy[:, 1], color='purple', linewidth=1.5, label='Trajectory')
    plt.plot(traj_xy[0, 0], traj_xy[0, 1], marker='s', color='green', markersize=2, label='Start')
    plt.plot(traj_xy[-1, 0], traj_xy[-1, 1], marker='s', color='red', markersize=2, label='End')


  # maps_rgb是一个HWC格式的图像数组
    plt.axis('off')    # 不显示坐标轴（可选）
    plt.show()
visualize(batch,idx)

In [ ]:
aux_info,_,scaled_input = model.pre_vae(batch)

In [ ]:
import torch
from src.models.dm_mlp import MLPTimeEmbedding
t = torch.randint(0,1000,(8,),)
model = MLPTimeEmbedding()
time = model(t)
time.shape

In [ ]:
model